# 한빛아카데미 - 새로나온 책 10페이지 크롤링
- url =https://www.hanbit.co.kr/academy/books/new_book_list.html
- 제목,저자,출간일,쪽수,가격,역자 정보 가져오기


In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm.notebook import tqdm


In [6]:
base_url = 'https://www.hanbit.co.kr'
sub_url = '/academy/books/new_book_list.html'
url = base_url + sub_url + '?page=' + str(1)
req = requests.get(url)
soup = BeautifulSoup(req.text,'html.parser')#문법의 의미를 분석하라 = parser

### 1. 찾으려고 하는 데이터 목록 갖고오기

In [7]:
lis = soup.select('li.sub_book_list')
len(lis)

20

### 2.하나의 원하는 정보 가져오기

In [8]:
li = lis[0]
href = li.find('a')['href']
href

'/academy/books/book_view.html?p_code=B8672666879'

- sub page로 이동해서 원하는 정보 추출

In [9]:
sub_req = requests.get(base_url + href)
sub_soup = BeautifulSoup(sub_req.text, 'html.parser')

In [13]:
title = sub_soup.select_one('.store_product_info_box > h3').get_text().strip()
title

'STEM CookBook, 문제 해결력을 키우는 기계설계'

In [15]:
info_lis = sub_soup.select('.info_list > li')
info_dict = {}
for li in info_lis:
  key = li.find('strong').get_text().split(' ')[0]
  val = li.find('span').get_text().strip()
  info_dict[key] = val
info_dict

{'ISBN': '9791156646150',
 '물류코드': '4615',
 '저자': '이상범',
 '출간': '2022-08-05',
 '페이지': '680 쪽'}

In [17]:
author = info_dict['저자']
try:
  translator = info_dict['번역']
except:
  translator = '-'
date = info_dict['출간']
page = int(info_dict['페이지'].split(' ')[0].replace(',',''))
author, translator, date, page

('이상범', '-', '2022-08-05', 680)

In [22]:
price = int(sub_soup.select_one('.pbr > strong').get_text().replace(',',''))
price

34000

### 3. 한 페이지 내 모든 정보 가져오기


In [23]:
lines= []
for li in lis:
  href = li.find('a')['href']
  sub_req = requests.get(base_url + href)
  sub_soup = BeautifulSoup(sub_req.text, 'html.parser')

  title = sub_soup.select_one('.store_product_info_box > h3').get_text().strip()
  info_lis = sub_soup.select('.info_list > li')
  info_dict = {}
  for li in info_lis:
    key = li.find('strong').get_text().split(' ')[0]
    val = li.find('span').get_text().strip()
    info_dict[key] = val

  author = info_dict['저자']
  try:
    translator = info_dict['번역']
  except:
    translator = '-'
  date = info_dict['출간']
  page = int(info_dict['페이지'].split(' ')[0].replace(',',''))
  price = int(sub_soup.select_one('.pbr > strong').get_text().replace(',',''))
  lines.append([title,author,translator,date,page,price])

### 4. 데이터 프레임으로 만들기

In [24]:
df = pd.DataFrame(lines, columns=['제목','저자','번역','출간일','쪽수','판매가'])
df

,제목,저자,번역,출간일,쪽수,판매가
0,"STEM CookBook, 문제 해결력을 키우는 기계설계",이상범,-,2022-08-05,680,34000
1,경영분석(6판),"장영광 , 송치승",-,2022-07-22,556,36000
2,"IT CookBook, 기초부터 시작하는 PLC : OMRON NX","정완보 , 이진호",-,2022-07-18,636,32000
3,"IT CookBook, 원리로 이해하는 전력전자공학(2판)","원충연 , 김영렬 , 이택기 , 류홍제 , 故정용채",-,2022-07-11,588,30000
4,"IT CookBook, TCP/IP 소켓 프로그래밍(2판)",김선우,-,2022-06-30,600,32000
5,"IT CookBook, 쉽게 배우는 데이터 통신과 컴퓨터 네트워크(3판)",박기현,-,2022-06-30,676,30000
6,"IT CookBook, 우분투 리눅스(3판)",이종원,-,2022-06-30,800,34000
7,거시경제학(3판),주상영,-,2022-06-30,640,36000
8,실용적 접근 방식의 경영학원론(10판),"Angelo Kinicki , Denise Breaux Soignet",김안드레아,2022-06-30,800,39000
9,"IT CookBook, C로 시작하는 컴퓨터 프로그래밍(4판)","문호석, 손명호",-,2022-06-26,432,23000


### 5. 10페이지의 정보로 만들기

In [25]:
lines = []
for page in tqdm(range(1,11)):
  url = base_url + sub_url + '?page=' + str(page)
  req = requests.get(url)
  soup = BeautifulSoup(req.text,'html.parser')
  lis = soup.select('li.sub_book_list')

  for li in lis:
    href = li.find('a')['href']
    sub_req = requests.get(base_url + href)
    sub_soup = BeautifulSoup(sub_req.text, 'html.parser')

    title = sub_soup.select_one('.store_product_info_box > h3').get_text().strip()
    info_lis = sub_soup.select('.info_list > li')
    info_dict = {}
    for li in info_lis:
      key = li.find('strong').get_text().split(' ')[0]
      val = li.find('span').get_text().strip()
      info_dict[key] = val

    author = info_dict['저자']
    try:
      translator = info_dict['번역']
    except:
      translator = '-'
    date = info_dict['출간']
    page = int(info_dict['페이지'].split(' ')[0].replace(',',''))
    price = int(sub_soup.select_one('.pbr > strong').get_text().replace(',',''))
    lines.append([title,author,translator,date,page,price])

  0%|          | 0/10 [00:00<?, ?it/s]

### 6. 데이터프레임 저장

In [27]:
df = pd.DataFrame(lines, columns=['제목','저자','번역','출간일','쪽수','판매가'])
df.tail(10)

,제목,저자,번역,출간일,쪽수,판매가
190,"IT CookBook , MySQL로 배우는 데이터베이스 개론과 실습","박우창 , 남송휘 , 이현룡",-,2019-08-15,572,27000
191,그림으로 배우는 기초통계학(제4판),박상규,-,2019-08-10,410,27000
192,빅데이터 마케팅과의 융합을 위한 고객관계관리(제4판),"김승욱 , 강기두 , 김민철",-,2019-08-10,376,26000
193,"IT CookBook, HTML5 웹 프로그래밍 입문(3판)",윤인성,-,2019-07-20,488,26000
194,"IT CookBook, 컴퓨터 활용과 실습 2016",김주영,-,2019-07-15,520,24000
195,"IT Cookbook, 처음 만나는 전자회로",황형수,-,2019-07-10,440,24000
196,스트랭 미분방정식과 선형대수학,Gilbert Strang,한빛수학교재연구소,2019-07-09,552,30000
197,"IT CookBook, 네트워크 개론(3판)",진혜진,-,2019-07-05,516,25000
198,"IT CookBook, 텐서플로로 배우는 딥러닝","박혜정 , 석경하 , 심주용 , 황창하",-,2019-06-30,448,27000
199,"IT CookBook, 파워포인트 2016",김지연,-,2019-06-29,420,19000


In [28]:
df.to_csv('한빛신간도서(220808).csv',index=False)